<a href="https://colab.research.google.com/github/ramenwang/deep_learning_py/blob/master/Coursera/NLP/sarcasm_data_with_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import tensorflow as tf
import matplotlib.pyplot as plt

print(tf.__version__)

1.15.0


In [2]:
# get data
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2020-03-14 21:57:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c1f::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.06s   

2020-03-14 21:57:50 (89.3 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [5]:
urls, headlines, is_sarcastic = [], [], []
with open('/tmp/sarcasm.json', 'r') as jsonfile:
    reader = json.load(jsonfile)
    for row in reader:
        urls.append(row.get('article_link'))
        headlines.append(row.get('headline'))
        is_sarcastic.append(row.get('is_sarcastic'))

print(urls[0])
print(headlines[0])
print(is_sarcastic[0])

https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5
former versace store clerk sues over secret 'black code' for minority shoppers
0


In [0]:
voc_size = 10000
seq_len = 20
embedding_size = 16
oov_token = '<OOV>'

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer(num_words=voc_size)

In [0]:
train_size = 20000
train_dat, test_dat = headlines[:train_size], headlines[train_size:]
train_label, test_label = is_sarcastic[:train_size], is_sarcastic[train_size:]

# preprocessing
tokenizer.fit_on_texts(train_dat)
word_index = tokenizer.word_index
train_seq = tokenizer.texts_to_sequences(train_dat)
test_seq = tokenizer.texts_to_sequences(test_dat)
train_padded = pad_sequences(train_seq, maxlen=seq_len, padding='post', truncating='post')
test_padded = pad_sequences(test_seq, maxlen=seq_len, padding='post', truncating='post')

In [12]:
# build up model graph
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=voc_size, output_dim=16, input_length=seq_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 24)                1560      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 174,129
Trainable params: 174,129
Non-trainable params: 0
_________________________________________________________________


In [13]:
run_time = model.fit(x = train_padded,
                     y = train_label,
                     epochs = 50,
                     validation_data = (test_padded, test_label))

Train on 20000 samples, validate on 6709 samples
Epoch 1/50
20000/20000 [==============================] - 51s 3ms/sample - loss: 0.4080 - acc: 0.7977 - val_loss: 0.3601 - val_acc: 0.8398
Epoch 2/50
20000/20000 [==============================] - 50s 2ms/sample - loss: 0.2242 - acc: 0.9094 - val_loss: 0.3529 - val_acc: 0.8517
Epoch 3/50
20000/20000 [==============================] - 50s 2ms/sample - loss: 0.1574 - acc: 0.9386 - val_loss: 0.3957 - val_acc: 0.8489
Epoch 4/50
20000/20000 [==============================] - 50s 2ms/sample - loss: 0.1107 - acc: 0.9597 - val_loss: 0.4895 - val_acc: 0.8414
Epoch 5/50
20000/20000 [==============================] - 49s 2ms/sample - loss: 0.0774 - acc: 0.9720 - val_loss: 0.5525 - val_acc: 0.8390
Epoch 6/50
20000/20000 [==============================] - 49s 2ms/sample - loss: 0.0475 - acc: 0.9844 - val_loss: 0.5915 - val_acc: 0.8386
Epoch 7/50
20000/20000 [==============================] - 49s 2ms/sample - loss: 0.0304 - acc: 0.9905 - val_loss: 0.8